In [1]:

#!pip install transformers librosa soundfile numpy"<"2 torch torchaudio fast-tsp kmedoids


# Setup

In [1]:
import os
%load_ext autoreload
%autoreload 2

from clap_slice import AudioOrderer, CLAPWrapper

import torch
torch.set_grad_enabled(False)

clap = CLAPWrapper()

objc[77660]: Class AVFFrameReceiver is implemented in both /usr/local/Cellar/ffmpeg/6.1_1/lib/libavdevice.60.3.100.dylib (0x16e697378) and /Users/damian/2.current/clapSlice/venv/lib/python3.12/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x1867b23b0). One of the two will be used. Which one is undefined.
objc[77660]: Class AVFAudioReceiver is implemented in both /usr/local/Cellar/ffmpeg/6.1_1/lib/libavdevice.60.3.100.dylib (0x16e6973c8) and /Users/damian/2.current/clapSlice/venv/lib/python3.12/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x1867b2400). One of the two will be used. Which one is undefined.


initilizing CLAP


# Run

In [2]:
filename = '/Users/damian/2.current/clapSlice/outputs/Anderson .Paak - Come Down (Official Video).mp3'
bpm = 98.4 #97.62396675

audio_orderer = AudioOrderer(clap=clap, source_audio_path=filename, bpm=bpm)

loaded waveform with shape torch.Size([2, 8381475]) , sampling rate 48000


In [11]:
chunk_beats = 8
window_width = 0
sort_order = audio_orderer.make_order(chunk_beats=chunk_beats, preserve_start_and_end=True)
sort_order

  0%|          | 0/35 [00:00<?, ?it/s]

computing distance matrix
computing route


AudioOrdering(source_audio='/Users/damian/2.current/clapSlice/outputs/Anderson .Paak - Come Down (Official Video).mp3', bpm=98.4, chunk_beats=8, sort_order=tensor([ 0,  1, 31, 30, 17, 29, 27, 15, 26, 14, 25, 13, 24, 23, 11, 10, 20, 12,
        19, 21, 22,  9, 18, 32, 16, 28,  7,  8,  6,  5,  3,  4,  2, 33, 34]), window_width=0)

In [12]:
from clap_slice.audio_ordering_candidates_registry import save_candidate
#save_candidate(sort_order)

'./audio_ordering_candidates_registry/Anderson .Paak - Come Down (Official Video).mp3/candidate_1.pkl'

In [13]:
from clap_slice import SmearModifier

smear_modifiers = [
    SmearModifier(smear_width=1, spread=4, match_embedding=clap.get_text_features("vocal, song, emotional singing")),
    SmearModifier(smear_width=2, spread=1, match_embedding=clap.get_text_features("instrumental"))
]

In [14]:


audio_ordering_result = audio_orderer.apply_order(
    audio_ordering=sort_order,
    smear_width=2,
    spread=0,
    wrap_mode='bleed',
    save=True,
    smear_modifiers=smear_modifiers,
    smooth_smear_modifiers=True
)
!open outputs
#Audio(result.output_audio, rate=audio_orderer.sampling_rate)
#print(ordered.shape)

[0, 1, 31, 30, 17, 29, 27, 15, 26, 14, 25, 13, 24, 23, 11, 10, 20, 12, 19, 21, 22, 9, 18, 32, 16, 28, 7, 8, 6, 5, 3, 4, 2, 33, 34]
smear width: 1.586115837097168  spread: 2.241652488708496 -> 2 2
smear width: 1.645477056503296  spread: 2.063568115234375 -> 2 2
smear width: 1.4874804019927979  spread: 2.5375595092773438 -> 1 3
smear width: 1.5476009845733643  spread: 2.357196569442749 -> 2 2
smear width: 1.5516889095306396  spread: 2.344932794570923 -> 2 2
smear width: 1.5988523960113525  spread: 2.2034430503845215 -> 2 2
smear width: 1.6059120893478394  spread: 2.1822636127471924 -> 2 2
smear width: 1.5897855758666992  spread: 2.2306430339813232 -> 2 2
smear width: 1.600127100944519  spread: 2.1996185779571533 -> 2 2
smear width: 1.585795283317566  spread: 2.242614269256592 -> 2 2
smear width: 1.5624415874481201  spread: 2.3126754760742188 -> 2 2
smear width: 1.5310919284820557  spread: 2.406723976135254 -> 2 2
smear width: 1.6349599361419678  spread: 2.0951199531555176 -> 2 2
smear wi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Video

In [ ]:
from clap_slice import VideoChunkCache
from clap_slice import VideoWriter, apply_audio_smear_to_video
from pytorchvideo.data.encoded_video import EncodedVideo

video_in_filename = 'outputs/Dua Lipa - Be The One (Official Music Video) (480p_25fps_H264-128kbit_AAC).mp4'

video = EncodedVideo.from_path(video_in_filename, decode_audio=False)
video_chunk_cache = VideoChunkCache(video=video,
                                    chunk_size_seconds=audio_orderer.get_chunk_size_seconds(sort_order.chunk_beats),
                                    max_cache_size=30)

In [ ]:
blend_mode = 'max'
video_writer = VideoWriter(output_path=f'outputs/Dua Lipa - Be The One (Official Music Video).mp3-sorted-bpm87.5-cb2-ww0-smeared-dyn.wav.mp3.{blend_mode}-sqrt2.mp4', fps=25, width=720, height=480)

apply_audio_smear_to_video(video_chunk_cache,
                           video_writer,
                           smear_details=audio_ordering_result.smear_details,
                           blend_mode=blend_mode,
                           max_chunks_to_write=None)

video_writer.close()

normalization_scaling: 0.4870553945448325


  0%|          | 0/152 [00:00<?, ?it/s]

loaded chunk 22
loaded chunk 11
loaded chunk 23
loaded chunk 12
loaded chunk 24
loaded chunk 13
evicting chunk 0
loaded chunk 14
evicting chunk 1
loaded chunk 15
evicting chunk 2
loaded chunk 16
evicting chunk 3
loaded chunk 17
evicting chunk 4
loaded chunk 8
evicting chunk 5
loaded chunk 9
evicting chunk 91
loaded chunk 18
evicting chunk 97
loaded chunk 10
evicting chunk 40
loaded chunk 71
evicting chunk 92
loaded chunk 72
evicting chunk 35
loaded chunk 70
evicting chunk 41
loaded chunk 73
evicting chunk 93
loaded chunk 74
evicting chunk 36
loaded chunk 64
evicting chunk 96
loaded chunk 75
evicting chunk 94
loaded chunk 65
evicting chunk 37
loaded chunk 66
evicting chunk 95
loaded chunk 76
evicting chunk 38
loaded chunk 67
evicting chunk 39
loaded chunk 62
evicting chunk 24
loaded chunk 77
evicting chunk 13
loaded chunk 68
evicting chunk 23
loaded chunk 63
evicting chunk 14
loaded chunk 78
evicting chunk 6
loaded chunk 69
evicting chunk 7
loaded chunk 23
evicting chunk 8
loaded chunk 